In [3]:
use_heatmap_data_weights = heatmap_data_weights
use_locations_weights = locations_weights
import pickle
with open('use_heatmap_data_weights.pkl', 'wb') as f:
    pickle.dump(use_heatmap_data_weights, f)
with open('use_locations_weights.pkl', 'wb') as f:
    pickle.dump(use_locations_weights, f)

In [1]:
#Hospital info has state and county
#Merge based on?
import pandas as pd

# load the two csv files into dataframes
df_updated = pd.read_csv('updated_with_state_icu_normalized.csv')
df_weights = pd.read_csv('combined_data_weights.csv')

# normalize the County and State column names in both dataframes
df_updated = df_updated.rename(columns={'County': 'county', 'State': 'state_name'})
df_weights = df_weights.rename(columns={'county': 'county', 'state_name': 'state_name'})

# define the columns we're interested in
columns_to_add = [
    'normalized_covid', 'normalized_sepsis', 'normalized_drowning', 
    'normalized_vaccination', 'normalized_flu', 'normalized_pneumonia',
    'normalized_literacy', 'normalized_seniors', 'normalized_smoking', 
    'normalized_copd', 'normalized_health_insurance', 'normalized_incomes'
]

# make sure all those columns exist in the updated dataframe
for column in columns_to_add:
    if column not in df_updated.columns:
        df_updated[column] = None

# iterate through each row in the updated dataframe
for idx, row in df_updated.iterrows():
    # find the corresponding row in the weights dataframe
    matching_row = df_weights[(df_weights['county'] == row['county']) & (df_weights['state_name'] == row['state_name'])]
    
    # if there is a match, update the values for the specified columns
    if not matching_row.empty:
        for column in columns_to_add:
            df_updated.at[idx, column] = matching_row[column].values[0]

# save the updated dataframe
df_updated.to_csv('hospitals_and_scores.csv', index=False)


In [2]:
import pandas as pd

def process_dataframe(scale):
    # Function to convert list
    
    def convert_list(input_list):
        # Remove 12 and 13 if they exist
        input_list = [i for i in input_list if i not in [12, 13]]
        
        # Create a list of zeros with a length of 12
        result_list = [0]*12
        
        # Set the index to 1 if it appears in the input_list
        for i in input_list:
            if i < 12:
                result_list[i] = 1
                
        return result_list
    
    scale_list = convert_list(scale)

    # Convert csv to a dataframe
    csv_path = 'hospitals_and_scores.csv'
    df = pd.read_csv(csv_path)

    # The list of column names in order
    column_names = [
        'normalized_smoking',
        'normalized_copd',
        'normalized_covid',
        'normalized_drowning',
        'normalized_sepsis',
        'normalized_flu',
        'normalized_pneumonia',
        'normalized_vaccination',
        'normalized_literacy',
        'normalized_incomes',
        'normalized_seniors',
        'normalized_health_insurance'
    ]

    # Weights for the columns
    weight_list = [0.07796371, 0.07551387, 0.14111341, 0.05940721, 0.15844383, 0.01946195, 0.14150974, 0.10762911, 0.09942487, 0.06826268, 0.05126962, 0.2]
    #df['normalized_health_insurance'] = 1 - df['normalized_health_insurance']
    # Iterate over each column name and its index
    for i, column in enumerate(column_names):
        # Multiply the column by its corresponding weight and scale
        df[column] = df[column] * weight_list[i] * scale_list[i]

    # Create a new 'score' column that is the sum of all the processed columns
    df['score'] = df[column_names].sum(axis=1)

    df = df.sort_values('score', ascending=False)

    # Remove duplicates based on 'Temp Locations 2' column, keeping the first occurrence
    df = df.drop_duplicates(subset='Temp Locations 2', keep='first')

    # List of columns to delete
    columns_to_delete = ['Rank', 'Current/Most Recent Study', 'Status', 'Study Results', 
                         'Interventions', 'Outcome Measures', 'Age', 'Phases', 'Enrollment', 
                         'Study Type', 'Study Designs', 'Completion Date', 'URL', 
                         'Other Locations', 'Latitude', 'Longitude']

    # Delete the columns
    df = df.drop(columns=columns_to_delete)

    df.to_csv('rankedlocations.csv')
    #return df


In [3]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import numpy as np
from weight_optimization import calculate_weights
from dataloader import load_data
from weight_optimization_rf import calculate_weights_rf
from branca.element import Template, MacroElement
import json
import pickle
import cProfile
import re

def preprocess_data(county_coordinates, smoking_data, copd_data, covid_data, sepsis_data, drowning_data, vaccination_data, flu_data, pneumonia_data, literacy_rates, incomes, age, seniors, health_insurance):
    state_abbreviations = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
    }
    merged_covid = pd.merge(covid_data, county_coordinates, left_on='fips', right_on='county_fips')
    merged_covid['cases_per_population'] = merged_covid['cases'] / merged_covid['population']
    merged_covid.dropna(subset=['lat', 'lng', 'cases_per_population'], inplace=True)
    filtered_covid = merged_covid[(merged_covid['state_name'] != 'Alaska') & (merged_covid['state_name'] != 'Hawaii') & (merged_covid['lat'] < 60)]
    min_covid, max_covid = filtered_covid['cases_per_population'].min(), filtered_covid['cases_per_population'].max()
    normalized_covid = filtered_covid.copy()
    normalized_covid['normalized_covid'] = (filtered_covid['cases_per_population'] - min_covid) / (max_covid - min_covid)

    merged_smoking = pd.merge(county_coordinates, smoking_data, left_on='state_name', right_on='LocationDesc', how='left')
    merged_smoking['Data_Value'].fillna(merged_smoking.groupby('state_name')['Data_Value'].transform('mean'), inplace=True)
    filtered_smoking = merged_smoking[(merged_smoking['state_name'] != 'Alaska') & (merged_smoking['state_name'] != 'Hawaii') & (merged_smoking['lat'] < 60)]
    min_smoking, max_smoking = filtered_smoking['Data_Value'].min(), filtered_smoking['Data_Value'].max()
    normalized_smoking = filtered_smoking.copy()
    normalized_smoking['normalized_smoking'] = (filtered_smoking['Data_Value'] - min_smoking) / (max_smoking - min_smoking)

    merged_copd = pd.merge(copd_data, county_coordinates, left_on='LocationID', right_on='county_fips')
    merged_copd.dropna(subset=['lat', 'lng', 'Percent_COPD'], inplace=True)
    filtered_copd = merged_copd[(merged_copd['state_name'] != 'Alaska') & (merged_copd['state_name'] != 'Hawaii') & (merged_copd['lat'] < 60)]
    min_copd, max_copd = 3.2, 15.5
    normalized_copd = filtered_copd.copy()
    normalized_copd['normalized_copd'] = (filtered_copd['Percent_COPD'] - min_copd) / (max_copd - min_copd)

    sepsis_data['STATE_FULL'] = sepsis_data['STATE'].apply(lambda x: state_abbreviations.get(x))
    merged_sepsis = pd.merge(county_coordinates, sepsis_data, left_on='state_name', right_on='STATE_FULL', how='left')
    min_sepsis, max_sepsis = merged_sepsis['RATE'].min(), merged_sepsis['RATE'].max()
    normalized_sepsis = merged_sepsis.copy()
    normalized_sepsis['normalized_sepsis'] = (merged_sepsis['RATE'] - min_sepsis) / (max_sepsis - min_sepsis)

    drowning_data['STATE_FULL'] = drowning_data['State'].apply(lambda x: state_abbreviations.get(x))
    merged_drowning = pd.merge(county_coordinates, drowning_data, left_on='state_name', right_on='STATE_FULL', how='left')
    min_drowning, max_drowning = merged_drowning['Dd'].min(), merged_drowning['Dd'].max()
    normalized_drowning = merged_drowning.copy()
    normalized_drowning['normalized_drowning'] = (merged_drowning['Dd'] - min_drowning) / (max_drowning - min_drowning)

    merged_vaccination = pd.merge(county_coordinates, vaccination_data, left_on='state_name', right_on='Location', how='left')
    min_vaccination, max_vaccination = merged_vaccination['Flu Vaccination Rate'].min(), merged_vaccination['Flu Vaccination Rate'].max()
    normalized_vaccination = merged_vaccination.copy()
    normalized_vaccination['normalized_vaccination'] = (merged_vaccination['Flu Vaccination Rate'] - min_vaccination) / (max_vaccination - min_vaccination)

    merged_flu = pd.merge(county_coordinates, flu_data, left_on='state_name', right_on='STATENAME', how='left')
    min_flu, max_flu = merged_flu['ACTIVITY_LEVEL'].min(), merged_flu['ACTIVITY_LEVEL'].max()
    normalized_flu = merged_flu.copy()
    normalized_flu['normalized_flu'] = (merged_flu['ACTIVITY_LEVEL'] - min_flu) / (max_flu - min_flu)

    pneumonia_data['state_name'] = pneumonia_data['STATE'].apply(lambda x: state_abbreviations.get(x))
    merged_pneumonia = pd.merge(county_coordinates, pneumonia_data, on='state_name', how='left')
    min_pneumonia, max_pneumonia = merged_pneumonia['RATE'].min(), merged_pneumonia['RATE'].max()
    normalized_pneumonia = merged_pneumonia.copy()
    normalized_pneumonia['normalized_pneumonia'] = (merged_pneumonia['RATE'] - min_pneumonia) / (max_pneumonia - min_pneumonia)


    #CHRIS: Additional Factor 1
    literacy_rates["Literacy Rate"] = literacy_rates["Literacy Rate"].str.replace("%", "").astype(float)
    merged_literacy = pd.merge(county_coordinates, literacy_rates, left_on='state_name', right_on='State', how='left')
    min_literacy, max_literacy = merged_literacy['Literacy Rate'].min(), merged_literacy['Literacy Rate'].max()
    normalized_literacy = merged_literacy.copy()
    normalized_literacy['normalized_literacy'] = (merged_literacy['Literacy Rate'] - min_literacy) / (max_literacy - min_literacy)
    normalized_literacy['normalized_literacy'] = 1 - normalized_literacy['normalized_literacy']

    #CHRIS: Additional factor 2
    merged_seniors = pd.merge(county_coordinates, seniors, left_on='state_name', right_on='State', how='left')
    min_seniors, max_seniors = merged_seniors['Percent'].min(), merged_seniors['Percent'].max()
    normalized_seniors = merged_seniors.copy()
    normalized_seniors['normalized_seniors'] = (merged_seniors['Percent'] - min_seniors) / (max_seniors - min_seniors)
    normalized_seniors['normalized_seniors'] = 1 - normalized_seniors['normalized_seniors']


    #CHRIS: Additional Factor 3
    colnames = incomes.columns.tolist()
    colnames[0] = 'County'
    colnames[1] = 'Income'
    incomes.columns = colnames
    merged_incomes = pd.merge(incomes, county_coordinates, left_on='County', right_on='county')
    merged_incomes.dropna(subset=['lat', 'lng', 'Income'], inplace=True)
    merged_incomes['Income'] = merged_incomes['Income'].str.replace(',', '').astype(float)
    filtered_incomes = merged_incomes[(merged_incomes['state_name'] != 'Alaska') & (merged_incomes['state_name'] != 'Hawaii') & (merged_incomes['lat'] < 60)]
    min_incomes, max_incomes = merged_incomes['Income'].min(), merged_incomes['Income'].max()
    normalized_incomes = filtered_incomes.copy()
    normalized_incomes['normalized_incomes'] = (filtered_incomes['Income'] - min_incomes) / (max_incomes - min_incomes)
    normalized_incomes['normalized_incomes'] = 1 - normalized_incomes['normalized_incomes']


    #CHRIS: Additional Factor 4
    normalized_health_insurance = health_insurance.rename(columns={'Percent': 'normalized_health_insurance'})
    min_insurance, max_insurance = normalized_health_insurance['normalized_health_insurance'].min(), normalized_health_insurance['normalized_health_insurance'].max()
    normalized_health_insurance['normalized_health_insurance'] = (normalized_health_insurance['normalized_health_insurance'] - min_insurance)/(max_insurance - min_insurance)


    

    combined_data = pd.merge(normalized_smoking, normalized_copd, on=['lat', 'lng', 'state_name'], suffixes=('_smoking', '_copd'))
    combined_data = pd.merge(combined_data, normalized_covid, on=['lat', 'lng', 'state_name'])
    combined_data = pd.merge(combined_data, normalized_sepsis[['county_fips', 'normalized_sepsis']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_drowning[['county_fips', 'normalized_drowning']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_vaccination[['county_fips', 'normalized_vaccination']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_flu[['county_fips', 'normalized_flu']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_pneumonia[['county_fips', 'normalized_pneumonia']], on='county_fips')

    #CHRIS: Merging additional factors
    combined_data = pd.merge(combined_data, normalized_literacy[['county_fips', 'normalized_literacy']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_seniors[['county_fips', 'normalized_seniors']], on='county_fips')
    combined_data = pd.merge(combined_data, normalized_incomes, on=['lat', 'lng', 'state_name'])
    combined_data = combined_data[combined_data['date'] == '2023-01-01']
    combined_data = pd.merge(combined_data, normalized_health_insurance, on=["county_full_y", "state_name"], how="left")

    global temp
    temp = combined_data
    combined_data.to_csv('store_preprocessed.csv')
    return combined_data, temp

def weights(combined_data, weight_toggle):
    #weights, r2 = calculate_weights_rf('state_data_2.csv')
    #print('WEIGHTS', weights)
    #print('r2', r2)
    weights = [0.07796371, 0.07551387, 0.14111341, 0.05940721, 0.15844383, 0.01946195, 0.14150974, 0.10762911, 0.09942487, 0.06826268, 0.05126962]
    r2 = 0.8589019440879608
    #CHRIS: This function has minor modifications to add the new factors
    weights = np.append(weights, 0.2)
    weight_toggle_int = [int(i) for i in weight_toggle]
    weight_toggle_int = [x for x in weight_toggle_int if x not in [12, 13]]
    storeweights = weights.copy()
    storeweights*=0
    for i in weight_toggle_int:
        storeweights[i] = weights[i]
    

    #CHRIS: Code to scale
    #CHANGE FOR SCALING
    sum_of_weights = sum(storeweights[i] for i in range(len(storeweights)))
    print('Original sum of weights', sum_of_weights)
    scaling_factor = 1/sum_of_weights
    new_sum = sum_of_weights*scaling_factor
    print('New sum', new_sum)
    storeweights = [i * scaling_factor for i in storeweights]
    mainsum = sum(storeweights[i] for i in range(len(storeweights)))
    print('Main sum', mainsum)
    #End of code to scale

    combined_data['combined_weighted_value'] = (
        storeweights[0] * combined_data['normalized_smoking'] 
        + storeweights[1] * combined_data['normalized_copd']
        + storeweights[2] * combined_data['normalized_covid']
        + storeweights[3] * combined_data['normalized_drowning']
        + storeweights[4] * combined_data['normalized_sepsis']
        + storeweights[5] * combined_data['normalized_flu']
        + storeweights[6] * combined_data['normalized_pneumonia']
        + storeweights[7] * combined_data['normalized_vaccination']
        + storeweights[8] * combined_data['normalized_literacy']
        + storeweights[9] * combined_data['normalized_incomes']
        + storeweights[10] * combined_data['normalized_seniors']
        + storeweights[11] * combined_data['normalized_health_insurance']
    )
    global hold
    hold = combined_data
    combined_data.dropna(subset=['lat', 'lng', 'combined_weighted_value'], inplace=True)
    heatmap_data = combined_data[['lat', 'lng', 'combined_weighted_value']].values.tolist()
    df = pd.read_csv('updated_with_state_icu_normalized.csv')
    df['temp'] = df['Hospital Name'].str.lower()
    df = df.drop_duplicates(subset='temp')
    df = df.drop(columns='temp')
        
    df = df.drop_duplicates(subset='Hospital Name')
    locations = [(row['Latitude'], row['Longitude'], row['Hospital Name']) for _, row in df.iterrows()]

    #combined_data: combined_data_weights.csv
    #heatmap_data: print out and save list: heatmap_data_weights
    #locations: print out and save list: locations_weights
    #hold: hold: hold_weights.csv

    combined_data.to_csv('combined_data_weights.csv')
    hold.to_csv('hold_weights.csv')
    #print('HEATMAP_DATA', heatmap_data)
    #print('LOCATIONS', locations)
    #global locations_weights
    #locations_weights = locations
    #global heatmap_data_weights
    #heatmap_data_weights = heatmap_data

    return combined_data, heatmap_data, locations, hold
    

def create_usa_map():
    usa_center_latitude = 40
    usa_center_longitude = -98
    usa_map = folium.Map(location=[usa_center_latitude, usa_center_longitude], zoom_start=4)
    return usa_map

def state_data(combined_data):
    state_data = combined_data.groupby('state_name').mean().reset_index()
    sd = state_data[['state_name','normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 'normalized_flu', 'normalized_pneumonia', 'normalized_smoking', 'normalized_copd', 'normalized_covid']]
    vals = pd.read_csv('vals.csv')
    sd.to_csv('state_data.csv', index=False)
    sd['vals'] = df_vals['vals']
    return sd


def add_heatmap(usa_map, heatmap_data):
    custom_gradient = {
        0.0: '#0000FF', 
        0.7: '#3399FF',  
        0.89: '#66FF66',  
        0.94: '#FFFF00', 
        1.0: '#FF0000'    
    }
    HeatMap(heatmap_data, gradient=custom_gradient).add_to(usa_map)

# def add_marker_cluster(usa_map, locations, data):
#     marker_cluster = MarkerCluster().add_to(usa_map)
#     for lat, lon, name in locations:
#         row = data[data['Hospital Name'] == name]
#         if len(row) > 0:
#             popup_text = f"<h4>{name}</h4>"
#             if pd.notnull(row['URL'].iloc[0]):
#                 popup_text += f"""
#                     <div style="width: 300px;">
#                         <strong>Total Successful Studies - Historically:</strong> {row['Total Successful Trials'].iloc[0]}<br>
#                         <strong>ICU Beds in the County:</strong> {row['ICU Beds true'].iloc[0]}<br>
#                         <strong>Normalized Overall score:</strong> {row['Normalized Composite Scores'].iloc[0]}<br>
#                         <strong>Current/Most Recent Study:</strong> {row['Current/Most Recent Study'].iloc[0]}<br>
#                         <strong>Status:</strong> {row['Status'].iloc[0]}<br>
#                         <strong>Study Results:</strong> {row['Study Results'].iloc[0]}<br>
#                         <strong>Interventions:</strong> {row['Interventions'].iloc[0]}<br>
#                         <strong>Outcome Measures:</strong> {row['Outcome Measures'].iloc[0]}<br>
#                         <strong>Age:</strong> {row['Age'].iloc[0]}<br>
#                         <strong>Phases:</strong> {row['Phases'].iloc[0]}<br>
#                         <strong>Enrollment:</strong> {row['Enrollment'].iloc[0]}<br>
#                         <strong>Study Type:</strong> {row['Study Type'].iloc[0]}<br>
#                         <strong>Study Designs:</strong> {row['Study Designs'].iloc[0]}<br>
#                         <strong>Completion Date:</strong> {row['Completion Date'].iloc[0]}<br>
#                         <strong>URL:</strong> <a href="{row['URL'].iloc[0]}" target="_blank">Link</a><br>
#                         <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
#                         <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
#                     </div>
#                 """
#             else:
#                 popup_text += f"""
#                     <div style="width: 300px;">
#                         <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
#                 """
#                 if pd.notnull(row['Principal Investigator'].iloc[0]):
#                     popup_text += f"""
#                         <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
#                     """
#                 popup_text += "</div>"
                
#             folium.Marker(location=[lat, lon], popup=popup_text).add_to(marker_cluster)

# def add_marker_cluster(usa_map, locations, data):
#     marker_cluster = MarkerCluster().add_to(usa_map)
#     special_hospitals = [
#         'University of Alabama Medical Center, Birmingham',
#         'Houston Methodist Hospital, Houston',
#         'Mount Sinai Hospital, New York'
#     ]

#     # Manually add 'Singing River, Mississippi' and 'Kansas University Medical Centre (KUMC)'
#     folium.Marker(location=[30.4138542, -88.78126903], 
#                   popup="<h4>Singing River, Mississippi</h4>", 
#                   icon=folium.Icon(color='red')).add_to(usa_map)

#     folium.Marker(location=[39.0574, -94.6097], 
#                   popup="<h4>Kansas University Medical Centre (KUMC)</h4>", 
#                   icon=folium.Icon(color='red')).add_to(usa_map)

#     for lat, lon, name in locations:
#         if name in ['Singing River, Mississippi', 'Kansas University Medical Centre (KUMC)']:
#             continue  # skip these hospitals since we've manually added them

#         row = data[data['Hospital Name'] == name]
#         if len(row) > 0:
#             popup_text = f"<h4>{name}</h4>"
#             if pd.notnull(row['URL'].iloc[0]):
#                 popup_text += f"""
#                     <div style="width: 300px;">
#                         <strong>Total Successful Studies - Historically:</strong> {row['Total Successful Trials'].iloc[0]}<br>
#                         <strong>ICU Beds in the County:</strong> {row['ICU Beds true'].iloc[0]}<br>
#                         <strong>Normalized Overall score:</strong> {row['Normalized Composite Scores'].iloc[0]}<br>
#                         <strong>Current/Most Recent Study:</strong> {row['Current/Most Recent Study'].iloc[0]}<br>
#                         <strong>Status:</strong> {row['Status'].iloc[0]}<br>
#                         <strong>Study Results:</strong> {row['Study Results'].iloc[0]}<br>
#                         <strong>Interventions:</strong> {row['Interventions'].iloc[0]}<br>
#                         <strong>Outcome Measures:</strong> {row['Outcome Measures'].iloc[0]}<br>
#                         <strong>Age:</strong> {row['Age'].iloc[0]}<br>
#                         <strong>Phases:</strong> {row['Phases'].iloc[0]}<br>
#                         <strong>Enrollment:</strong> {row['Enrollment'].iloc[0]}<br>
#                         <strong>Study Type:</strong> {row['Study Type'].iloc[0]}<br>
#                         <strong>Study Designs:</strong> {row['Study Designs'].iloc[0]}<br>
#                         <strong>Completion Date:</strong> {row['Completion Date'].iloc[0]}<br>
#                         <strong>URL:</strong> <a href="{row['URL'].iloc[0]}" target="_blank">Link</a><br>
#                         <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
#                         <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
#                     </div>
#                 """
#             else:
#                 popup_text += f"""
#                     <div style="width: 300px;">
#                         <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
#                 """
#                 if pd.notnull(row['Principal Investigator'].iloc[0]):
#                     popup_text += f"""
#                         <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
#                     """
#                 popup_text += "</div>"
            
#             if name in special_hospitals:
#                 folium.Marker(location=[lat, lon], popup=popup_text, 
#                               icon=folium.Icon(color='red')).add_to(usa_map)
#             else: 
#                 folium.Marker(location=[lat, lon], popup=popup_text).add_to(marker_cluster)


def add_marker_cluster(usa_map, locations, data):
    marker_cluster = MarkerCluster().add_to(usa_map)
    special_hospitals = [
        'University of Alabama Medical Center, Birmingham',
        'Houston Methodist Hospital, Houston',
        'Mount Sinai Hospital, New York',
        'Singing River, Mississippi',
        'Kansas University Medical Centre (KUMC)'
    ]
    singing_river = False
    for lat, lon, name in locations:
        if name == 'Kansas University Medical Centre (KUMC)':
            singing_river = True
        row = data[data['Hospital Name'] == name]
        if len(row) > 0:
            popup_text = f"<h4>{name}</h4>"
            if pd.notnull(row['URL'].iloc[0]):
                popup_text += f"""
                    <div style="width: 300px;">
                        <strong>Total Successful Studies - Historically:</strong> {row['Total Successful Trials'].iloc[0]}<br>
                        <strong>ICU Beds in the County:</strong> {row['ICU Beds true'].iloc[0]}<br>
                        <strong>Normalized Overall score:</strong> {row['Normalized Composite Scores'].iloc[0]}<br>
                        <strong>Current/Most Recent Study:</strong> {row['Current/Most Recent Study'].iloc[0]}<br>
                        <strong>Status:</strong> {row['Status'].iloc[0]}<br>
                        <strong>Study Results:</strong> {row['Study Results'].iloc[0]}<br>
                        <strong>Interventions:</strong> {row['Interventions'].iloc[0]}<br>
                        <strong>Outcome Measures:</strong> {row['Outcome Measures'].iloc[0]}<br>
                        <strong>Age:</strong> {row['Age'].iloc[0]}<br>
                        <strong>Phases:</strong> {row['Phases'].iloc[0]}<br>
                        <strong>Enrollment:</strong> {row['Enrollment'].iloc[0]}<br>
                        <strong>Study Type:</strong> {row['Study Type'].iloc[0]}<br>
                        <strong>Study Designs:</strong> {row['Study Designs'].iloc[0]}<br>
                        <strong>Completion Date:</strong> {row['Completion Date'].iloc[0]}<br>
                        <strong>URL:</strong> <a href="{row['URL'].iloc[0]}" target="_blank">Link</a><br>
                        <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
                        <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
                    </div>
                """
            else:
                popup_text += f"""
                    <div style="width: 300px;">
                        <strong>Hospital Name:</strong> {row['Hospital Name'].iloc[0]}<br>
                """
                if pd.notnull(row['Principal Investigator'].iloc[0]):
                    popup_text += f"""
                        <strong>Principal Investigator:</strong> {row['Principal Investigator'].iloc[0]}
                    """
                popup_text += "</div>"
            
            if name in special_hospitals:
                folium.Marker(location=[lat, lon], popup=popup_text, 
                              icon=folium.Icon(color='red')).add_to(usa_map)
            else: 
                folium.Marker(location=[lat, lon], popup=popup_text).add_to(marker_cluster)
        if singing_river:
            print("'Singing River, Mississippi' is present in the locations.")
        else:
            print("'Singing River, Mississippi' is not present in the locations.")


import json
import requests

def add_state_borders_and_labels(usa_map):
    url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'

    # Load the geojson file
    geo_json_data = json.loads(requests.get(url).text)

    folium.GeoJson(
        geo_json_data,
        name='geojson'
    ).add_to(usa_map)

    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.0, 
                                'weight': 0.0}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.20, 
                                    'weight': 0.1}
    state_labels = folium.features.GeoJson(
        geo_json_data,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['name'],
            aliases=['State:'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
    usa_map.add_child(state_labels)
    usa_map.keep_in_front(state_labels)

def test_usa_map():
    create_usa_map()


def main(weight_toggle, marker_cluster_toggle, list_toggle):
    import time
    #CHRIS: Additional factors added
    #county_coordinates, smoking_data, copd_data, covid_data, sepsis_data, drowning_data, vaccination_data, flu_data, pneumonia_data, ards_centers, literacy_rates, incomes, age, seniors, health_insurance = load_data()
    #combined_data, temp = preprocess_data(county_coordinates, smoking_data, copd_data, covid_data, sepsis_data, drowning_data, vaccination_data, flu_data, pneumonia_data, literacy_rates, incomes, age, seniors, health_insurance)
    combined_data = pd.read_csv('store_preprocessed.csv')

    data, heatmap_data, locations, hold = weights(combined_data, weight_toggle)
    print("TIME 1")


    # with open('use_locations_weights.pkl', 'rb') as f:
    #     use_locations_weights = pickle.load(f)
    # with open('use_heatmap_data_weights.pkl', 'rb') as f:
    #     use_heatmap_data_weights = pickle.load(f)
    # print("TIME 2")
    # data, heatmap_data, locations, hold = pd.read_csv('combined_data_weights.csv'), use_heatmap_data_weights, use_locations_weights, pd.read_csv('hold_weights.csv')
    # print("TIME 3")

    #usa_map = cProfile.run('create_use_map()')
    
    

    usa_map = create_usa_map()
    print("TIME 4")
    #cProfile.run('test_usa_map()')
    add_heatmap(usa_map, heatmap_data)
    print("TIME 5")
    merged_final_with_investigator = pd.read_csv('updated_with_state_icu_normalized.csv')

    weight_toggle_list = [int(i) for i in weight_toggle]
    marker_cluster_toggle_list = 12 in weight_toggle_list
    list_toggle_list = 13 in weight_toggle_list

    process_dataframe(weight_toggle_list)
    #weight_toggle_list = [x for x in weight_toggle_list if x not in [8, 9]]

    print('WEIGHT LIST',weight_toggle_list)
    if marker_cluster_toggle_list:
        print("TIME 6")
        add_marker_cluster(usa_map, locations, merged_final_with_investigator)
    print("TIME 7")
    add_state_borders_and_labels(usa_map)
    
    
    class CustomMacroElement(MacroElement):
        def __init__(self, html, script, css):
            super().__init__()
            self._template = Template("""
                {% macro header(this, kwargs) %}
                    <style>
                    """ + css + """
                    </style>
                {% endmacro %}
                {% macro html(this, kwargs) %}
                    """ + html + """
                {% endmacro %}
                {% macro script(this, kwargs) %}
                    """ + script + """
                {% endmacro %}
            """)
    if list_toggle_list:
    # Define HTML, CSS and Javascript
        html = """
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">

        <div style="position: absolute; top: 10px; right: 20px; z-index: 9999; background-color: #fff; padding: 10px; border: 1px solid #ccc;">

        <img src="https://uploads-ssl.webflow.com/608bbb52742675860410dd77/608bbc12518ab6a69ad28a56_GEn1E%20Logo%20horizontal.png" alt="Logo" style="width: 125px; display: block; margin-left: auto; margin-right: auto;">
        <form id="checkbox-form">
            <fieldset>
                <label>
                    <input class="cb cb1" type="checkbox" name="check" value=0 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Smoking Data</span> 
                </label>
                <label>
                    <input class="cb cb2" type="checkbox" name="check" value=1 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>COPD Data</span> 
                </label>
                <label>
                    <input class="cb cb3" type="checkbox" name="check" value=2 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>COVID-19 Data</span> 
                </label>
                <label>
                    <input class="cb cb4" type="checkbox" name="check" value=3 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Drowning Data</span> 
                </label>
                <label>
                    <input class="cb cb5" type="checkbox" name="check" value=4 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Sepsis Data</span> 
                </label>
                <label>
                    <input class="cb cb6" type="checkbox" name="check" value=5 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Flu Data</span> 
                </label>
                <label>
                    <input class="cb cb7" type="checkbox" name="check" value=6 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Pneumonia Data</span> 
                </label>
                <label>
                    <input class="cb cb8" type="checkbox" name="check" value=7 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Vaccination Data</span> 
                </label>

            <label>
                <input class="cb cb9" type="checkbox" name="check" value=8 onclick="cbChange(this)"/>
                <i></i> 
                <span>Literacy Rates</span> 
            </label> 
            <label>
                <input class="cb cb10" type="checkbox" name="check" value=9 onclick="cbChange(this)"/>
                <i></i> 
                <span>Income</span> 
            </label> 
            <label>
                <input class="cb cb11" type="checkbox" name="check" value=10 onclick="cbChange(this)"/>
                <i></i> 
                <span>Age</span> 
            </label>    
            <label>
                <input class="cb cb12" type="checkbox" name="check" value=11 onclick="cbChange(this)"/>
                <i></i> 
                <span>Health Insurance</span> 
            </label>           
            <label>
                <input class="cb cb13" type="checkbox" name="check" value=12 onclick="cbChange(this)"/>
                <i></i> 
                <span>ARDS Trial Locations</span> 
            </label>         
            <label>
                <input class="cb cb14" type="checkbox" name="check" value=13 onclick="cbChange(this)"/>
                <i></i> 
                <span>Top Locations</span> 
            </label>    


                
            </fieldset>
        <div class="submit-area">
            <button type="submit">Compute</button>
        </div>
        <div id="loading-text" style="text-align: center; font-size: 16px; display: none;">GEn1E Ridge Processor</div>
        </form>

    </div> 
    <div style="position: fixed; bottom: 30px; right: 20px; z-index: 9999;">
    <img src="https://uploads-ssl.webflow.com/608bbb52742675860410dd77/647f8bf9ce5c07118bcd4b1e_County%20level%20scoring%20card%20LEGEND.png" alt="Logo" style="width: 214px;">
</div>


    <div id="sidebar">
    <button id="close" onclick="this.parentElement.style.display='none'">×</button>
        <h2>Top Ranked Locations</h2>
<ul id="myList">
    <li>
        <h3>University of Alabama at Birmingham</h3>
        <p>Score: 100 | PI: Carlo Waldemar et al. | Alabama</p>
        <p>6 previous successful trials | 471 ICU Beds</p>
    </li>
    <li>
        <h3>University of Kentucky Chandler Medical Center</h3>
        <p>Score: 98.38 | PI: Hubert Ballard | Kentucky</p>
        <p>3 previous successful trials | 260 ICU Beds</p>
    </li>
    <li>
        <h3>Helen Keller Hospital</h3>
        <p>Score: 92.56 | PI: Amy Lightner | Alabama</p>
        <p>1 previous successful trial | 22 ICU Beds</p>
    </li>
    <li>
        <h3>USA Women and Children’s Hospital</h3>
        <p>Score: 91.62 | PI: James Cummings | Alabama</p>
        <p>1 previous successful trial | 157 ICU Beds</p>
    </li>
    <li>
        <h3>Rockefeller Neuroscience Institute</h3>
        <p>Score: 90.74 | PI: Ali Rezai | West Virginia</p>
        <p>1 previous successful trial | 119 ICU Beds</p>
    </li>
    <li>
        <h3>Medical Affiliated Research Center</h3>
        <p>Score: 86.61 | PI: R Swerdloff | Alabama</p>
        <p>2 previous successful trials | 100 ICU Beds</p>
    </li>
</ul>
    </div>



    <script src="https://unpkg.com/@lottiefiles/lottie-player@latest/dist/lottie-player.js"></script>
        """
        script = """
        
        var checked_values = JSON.parse('""" + json.dumps(weight_toggle_list) + """');
        var initial_selected = [...checked_values];  // copy of the initial checked_values

        $('.cb').each(function() {
            var value = parseInt($(this).val());
            if (checked_values.includes(value)) {
                $(this).prop('checked', true);
            } else {
                $(this).prop('checked', false);
            }
        });

        // Disable the submit button initially
        $('#checkbox-form button[type="submit"]').prop('disabled', true);

        $('.cb').change(function() {
            // When a checkbox changes state, check if the selected checkboxes have changed
            var current_selected = [];
            $("input[type='checkbox']:checked").each(function(){
                current_selected.push(parseInt($(this).val()));
            });

            // If selected checkboxes have changed, enable the submit button. If not, disable it.
            if (JSON.stringify(current_selected.sort()) != JSON.stringify(initial_selected.sort())) {
                $('#checkbox-form button[type="submit"]').prop('disabled', false);
            } else {
                $('#checkbox-form button[type="submit"]').prop('disabled', true);
            }
        });

   var counter = 0;

    function loadingAnimation() {
        var dots = window.setInterval( function() {
            var wait = document.getElementById("loading-text");

            wait.innerHTML = "GEn1E Ridge Processor" + ".".repeat(counter+1);
            counter = (counter + 1) % 3;
        }, 1000);
        return dots;
    };


$('#checkbox-form').submit(function(e){
    e.preventDefault();
    var selected = [];
    $("input[type='checkbox']:checked").each(function(){
        selected.push($(this).val());
    });
    // Show loading text and start the animation
    $("#loading-text").css("display", "block");
    var dots = loadingAnimation(); // Start the loading animation
    $.ajax({
        url: '/checkboxes',
        type: 'post',
        contentType: 'application/json',
        data : JSON.stringify(selected),
        success: function(){
            window.location.href = "/result";
        },
        complete: function(){
            // Hide loading text and stop the animation
            $("#loading-text").css("display", "none");
            clearInterval(dots);
        }
    });
});

        """


        css = """
        *, *:before, *:after {
            box-sizing: border-box; 
        }

        html {
            font-size: 13.5px;
        }

        fieldset {
            display: block;
        }

        label {
            font-family: "Montserrat", sans-serif;
            font-size: 0.9rem;
            cursor: pointer;
            display: block;
            margin: 0.75em;
        }

        label > input {
            display: none;
        }

        label span {
            color: #6A759B;
        }

        label i {
            display: inline-block;
            width: 48px;
            height: 30px;
            border-radius: 15px;
            vertical-align: middle;
            transition: .25s .09s;
            position: relative;
            background: #deeff7;
        }

        label i:after {
            content: " ";
            display: block;
            width: 22.5px;
            height: 22.5px;
            top: 3.75px;
            left: 3.75px;
            border-radius: 50%;
            background: #fff;
            position: absolute;
            box-shadow: 1px 2px 4px 0 rgba(#000, .4);
            transition: .15s;
        }

        label > input:checked + i {
            background: #1094fb;
        }

        label > input:checked + i + span {
            color: #29316b;
        }

        label > input:checked + i:after {
            transform: translateX(18.75px);
        }

        button {
            font-family: "Montserrat", sans-serif;
            font-size: 0.9rem;
            color: #ffffff;
            background-color: #1094fb;
            border: none;
            padding: 10px;
            border-radius: 5px;
            cursor: pointer;
            display: block;
            margin: 10px auto;
        }
        #checkbox-form button[type="submit"]:disabled {
            background-color: lightgrey;
            cursor: not-allowed;
        }

    #sidebar {
        position: absolute;
        top: 10px;
        left: 10px;
        z-index: 9999;
        background: linear-gradient(135deg, #81d4fa 0%, #0288d1 100%);
        padding: 20px;
        border-radius: 15px;
        box-shadow: 0 4px 15px 0 rgba(129, 212, 250, 0.4);
        width: 382.5px; 
    }

    h2 {
        color: #fff;
        font-size: 24px;
        font-weight: 500;
        text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5);
    }

    #myList {
        list-style: none;
        padding-left: 0;
    }

    #myList li {
        padding: 15px 15px 15px; /* Reduced top padding */
        border-radius: 8px;
        background-color: rgba(255, 255, 255, 0.8);
        margin-bottom: 10px;
        transition: transform .2s;
    }

    #myList li:hover {
        transform: scale(1.02);
    }

    #myList li h3 {
        margin: 0 0 5px 0; 
        color: #0288d1;
        font-size: 18px;
    }

    #myList li p {
        margin: 0;
        color: #333;
        font-size: 14px;
    }
    #close {
    position: absolute;
    right: 10px;
    top: 10px;
    background: none;
    border: none;
    font-size: 20px;
    cursor: pointer;
    color: #fff;
}
        """
    else:
        html = """
        <div style="position: absolute; top: 10px; right: 20px; z-index: 9999; background-color: #fff; padding: 10px; border: 1px solid #ccc;">

        <img src="https://uploads-ssl.webflow.com/608bbb52742675860410dd77/608bbc12518ab6a69ad28a56_GEn1E%20Logo%20horizontal.png" alt="Logo" style="width: 125px; display: block; margin-left: auto; margin-right: auto;">
        <form id="checkbox-form">
            <fieldset>
                <label>
                    <input class="cb cb1" type="checkbox" name="check" value=0 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Smoking Data</span> 
                </label>
                <label>
                    <input class="cb cb2" type="checkbox" name="check" value=1 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>COPD Data</span> 
                </label>
                <label>
                    <input class="cb cb3" type="checkbox" name="check" value=2 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>COVID-19 Data</span> 
                </label>
                <label>
                    <input class="cb cb4" type="checkbox" name="check" value=3 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Drowning Data</span> 
                </label>
                <label>
                    <input class="cb cb5" type="checkbox" name="check" value=4 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Sepsis Data</span> 
                </label>
                <label>
                    <input class="cb cb6" type="checkbox" name="check" value=5 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Flu Data</span> 
                </label>
                <label>
                    <input class="cb cb7" type="checkbox" name="check" value=6 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Pneumonia Data</span> 
                </label>
                <label>
                    <input class="cb cb8" type="checkbox" name="check" value=7 onclick="cbChange(this)"/>
                    <i></i> 
                    <span>Vaccination Data</span> 
                </label>

            <label>
                <input class="cb cb9" type="checkbox" name="check" value=8 onclick="cbChange(this)"/>
                <i></i> 
                <span>Literacy Rates</span> 
            </label> 
            <label>
                <input class="cb cb10" type="checkbox" name="check" value=9 onclick="cbChange(this)"/>
                <i></i> 
                <span>Income</span> 
            </label> 
            <label>
                <input class="cb cb11" type="checkbox" name="check" value=10 onclick="cbChange(this)"/>
                <i></i> 
                <span>Age</span> 
            </label>    
            <label>
                <input class="cb cb12" type="checkbox" name="check" value=11 onclick="cbChange(this)"/>
                <i></i> 
                <span>Health Insurance</span> 
            </label>           
            <label>
                <input class="cb cb13" type="checkbox" name="check" value=12 onclick="cbChange(this)"/>
                <i></i> 
                <span>ARDS Trial Locations</span> 
            </label>         
            <label>
                <input class="cb cb14" type="checkbox" name="check" value=13 onclick="cbChange(this)"/>
                <i></i> 
                <span>Top Locations</span> 
            </label>    
            </fieldset>
        <div class="submit-area">
            <button type="submit">Compute</button>
        </div>
        <div id="loading-text" style="text-align: center; font-size: 16px; display: none;">GEn1E Ridge Processor</div>
        
        </form>
    </div> 
    <div style="position: fixed; bottom: 30px; left: 20px; z-index: 9999;">
    <img src="https://uploads-ssl.webflow.com/608bbb52742675860410dd77/647f8bf9ce5c07118bcd4b1e_County%20level%20scoring%20card%20LEGEND.png" alt="Logo" style="width: 214px;">
</div>

        """
        script = """
        
        var checked_values = JSON.parse('""" + json.dumps(weight_toggle_list) + """');
        var initial_selected = [...checked_values];  // copy of the initial checked_values

        $('.cb').each(function() {
            var value = parseInt($(this).val());
            if (checked_values.includes(value)) {
                $(this).prop('checked', true);
            } else {
                $(this).prop('checked', false);
            }
        });

        // Disable the submit button initially
        $('#checkbox-form button[type="submit"]').prop('disabled', true);

        $('.cb').change(function() {
            // When a checkbox changes state, check if the selected checkboxes have changed
            var current_selected = [];
            $("input[type='checkbox']:checked").each(function(){
                current_selected.push(parseInt($(this).val()));
            });

            // If selected checkboxes have changed, enable the submit button. If not, disable it.
            if (JSON.stringify(current_selected.sort()) != JSON.stringify(initial_selected.sort())) {
                $('#checkbox-form button[type="submit"]').prop('disabled', false);
            } else {
                $('#checkbox-form button[type="submit"]').prop('disabled', true);
            }
        });

    var counter = 0;

    function loadingAnimation() {
        var dots = window.setInterval( function() {
            var wait = document.getElementById("loading-text");

            wait.innerHTML = "GEn1E Ridge Processor" + ".".repeat(counter+1);
            counter = (counter + 1) % 3;
        }, 1000);
        return dots;
    };


$('#checkbox-form').submit(function(e){
    e.preventDefault();
    var selected = [];
    $("input[type='checkbox']:checked").each(function(){
        selected.push($(this).val());
    });
    // Show loading text and start the animation
    $("#loading-text").css("display", "block");
    var dots = loadingAnimation(); // Start the loading animation
    $.ajax({
        url: '/checkboxes',
        type: 'post',
        contentType: 'application/json',
        data : JSON.stringify(selected),
        success: function(){
            window.location.href = "/result";
        },
        complete: function(){
            // Hide loading text and stop the animation
            $("#loading-text").css("display", "none");
            clearInterval(dots);
        }
    });
});


        """


        css = """
        *, *:before, *:after {
            box-sizing: border-box; 
        }

        html {
            font-size: 13.5px;
        }

        fieldset {
            display: block;
        }

        label {
            font-family: "Montserrat", sans-serif;
            font-size: 0.9rem;
            cursor: pointer;
            display: block;
            margin: 0.75em;
        }

        label > input {
            display: none;
        }

        label span {
            color: #6A759B;
        }

        label i {
            display: inline-block;
            width: 48px;
            height: 30px;
            border-radius: 15px;
            vertical-align: middle;
            transition: .25s .09s;
            position: relative;
            background: #deeff7;
        }

        label i:after {
            content: " ";
            display: block;
            width: 22.5px;
            height: 22.5px;
            top: 3.75px;
            left: 3.75px;
            border-radius: 50%;
            background: #fff;
            position: absolute;
            box-shadow: 1px 2px 4px 0 rgba(#000, .4);
            transition: .15s;
        }

        label > input:checked + i {
            background: #1094fb;
        }

        label > input:checked + i + span {
            color: #29316b;
        }

        label > input:checked + i:after {
            transform: translateX(18.75px);
        }

        button {
            font-family: "Montserrat", sans-serif;
            font-size: 0.9rem;
            color: #ffffff;
            background-color: #1094fb;
            border: none;
            padding: 10px;
            border-radius: 5px;
            cursor: pointer;
            display: block;
            margin: 10px auto;
        }
        #checkbox-form button[type="submit"]:disabled {
            background-color: lightgrey;
            cursor: not-allowed;
        }
        
        """        

    # Create custom Macro Element
    macro = CustomMacroElement(html, script, css)

    # Add custom Macro Element to map
    usa_map.get_root().add_child(macro)

    usa_map.save('togglenow3.html')
    print('TIME 8')


In [ ]:
#Add logic
#Change loading
from flask import Flask, request, redirect, send_from_directory
import time
import json
from flask import Flask, request, render_template

#app = Flask(__name__)
app = Flask(__name__, template_folder='/Users/sumeet/Documents/GEN1E')
@app.route('/')
def index():
    return render_template('starting_map_loader_revamped.html')  # assuming the html file is named 'checkbox.html'

@app.route('/checkboxes', methods=['POST'])
def checkboxes():
    indexes = request.get_json()
    print(indexes)  # print out the received indexes
    #time.sleep(10)  # pause for 10 seconds


    main(indexes, indexes, indexes)  # assuming you have a function named 'create_html' that creates the new HTML file
    #time.sleep(10)
    return json.dumps({'success':True}), 200, {'ContentType':'application/json'}

@app.route('/result')
def result():
    return send_from_directory('/Users/sumeet/Documents/GEN1E', 'togglenow3.html')  # send the file from the server

# if __name__ == '__main__':
#     app.run(debug=True)
import threading

def run_app():
    app.run(threaded=True)

t = threading.Thread(target=run_app)
t.start()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jul/2023 13:34:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2023 13:34:20] "GET /resources/demos/style.css HTTP/1.1" 404 -


['1', '4', '6', '10', '12']
Original sum of weights 0.42673706
New sum 1.0
Main sum 1.0
TIME 1
TIME 4
TIME 5
WEIGHT LIST [1, 4, 6, 10, 12]
TIME 6
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in the locations.
'Singing River, Mississippi' is not present in t

127.0.0.1 - - [27/Jul/2023 13:34:27] "POST /checkboxes HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2023 13:34:27] "GET /result HTTP/1.1" 200 -


TIME 8


In [5]:
import pandas as pd

# Load your csv file into a pandas DataFrame
df = pd.read_csv('rankedlocations.csv')

# Assuming your DataFrame is sorted by 'score', you can add a 'ranking' column like this:
df['ranking'] = range(1, len(df) + 1)

# Create a list of locations you're interested in
locations = [
    'University of Alabama Medical Center',
    'Singing River, Mississippi',
    'Bringham Young, Massachusetts',
    'Kansas University Medical Centre (KUMC)',
    'University of Cincinnati',
    'Houston Methodist Hospital',
    'UVA Health Center',
    'Washington University School of Medicine'
    'North Shore University Hospital'
]

# Use the isin() function to filter rows whose 'Temp Locations 2' value is in the list
filtered_df = df[df['Temp Locations 2'].isin(locations)]

# Print out the 'Temp Locations 2' and 'ranking' columns for these rows
print(filtered_df[['Temp Locations 2', 'ranking']])


                            Temp Locations 2  ranking
1       University of Alabama Medical Center        2
4                 Singing River, Mississippi        5
26                Houston Methodist Hospital       27
82                  University of Cincinnati       83
86   Kansas University Medical Centre (KUMC)       87
108                        UVA Health Center      109
136            Bringham Young, Massachusetts      137


In [6]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

best_rank = len(df) + 1
best_weights = None

# calculate and print the rank when all weights are 1
df['score'] = df[score_columns].sum(axis=1)
df['ranking'] = df['score'].rank(method='min', ascending=False)
uc_rank_all_one = df.loc[df['Temp Locations 2'] == 'North Shore University Hospital', 'ranking'].iloc[0]
print(f"Rank of University of Cincinnati when all weights are 1: {uc_rank_all_one}")

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 4 weights are 1
    if sum(weights) < 4:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the rank of University of Cincinnati
    uc_rank = df_copy.loc[df_copy['Temp Locations 2'] == 'University of Cincinnati', 'ranking'].iloc[0]
    # if this is the best rank we've found so far, store the weights
    if uc_rank < best_rank:
        best_rank = uc_rank
        best_weights = weights

# print the best weights and the corresponding rank
print(f"Best weights: {best_weights}")
print(f"Rank of University of Cincinnati with best weights: {best_rank}")


Rank of University of Cincinnati when all weights are 1: 40.0
Best weights: (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1)
Rank of University of Cincinnati with best weights: 4.0


In [7]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

# list of locations of interest
locations = ['University of Alabama Medical Center', 'Singing River, Mississippi', 'Bringham Young, Massachusetts',
             'Kansas University Medical Centre (KUMC)', 'University of Cincinnati', 'Houston Methodist Hospital',
             'UVA Health Center', 'Washington University School of Medicine']

best_count = 0
best_weights = None

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 4 weights are 1
    if sum(weights) < 4:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the number of locations with rank less than 10
    count = sum(df_copy.loc[df_copy['Temp Locations 2'].isin(locations), 'ranking'] < 20)
    # if this is the best count we've found so far, store the weights
    if count > best_count:
        best_count = count
        best_weights = weights

# print the best weights and the corresponding count
print(f"Best weights: {best_weights}")
print(f"Number of locations with rank less than 50 using best weights: {best_count}")


Best weights: (0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1)
Number of locations with rank less than 50 using best weights: 3


In [8]:
# apply the best weights to the DataFrame
df_best = df.copy()
for column, weight in zip(score_columns, best_weights):
    df_best[column] *= weight
# calculate the new score
df_best['score'] = df_best[score_columns].sum(axis=1)
# rank the rows by the new score
df_best['ranking'] = df_best['score'].rank(method='min', ascending=False)
# print the rankings of the selected locations
for location in locations:
    rank = df_best.loc[df_best['Temp Locations 2'] == location, 'ranking'].iloc[0]
    print(f"Rank of {location} with best weights: {rank}")



Rank of University of Alabama Medical Center with best weights: 7.0
Rank of Singing River, Mississippi with best weights: 17.0
Rank of Bringham Young, Massachusetts with best weights: 208.0
Rank of Kansas University Medical Centre (KUMC) with best weights: 65.0
Rank of University of Cincinnati with best weights: 5.0
Rank of Houston Methodist Hospital with best weights: 67.0
Rank of UVA Health Center with best weights: 192.0
Rank of Washington University School of Medicine with best weights: 29.0


In [9]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

# list of locations of interest
locations = ['University of Alabama Medical Center', 'Singing River, Mississippi', 'Bringham Young, Massachusetts',
             'Kansas University Medical Centre (KUMC)', 'University of Cincinnati', 'Houston Methodist Hospital',
             'UVA Health Center', 'Washington University School of Medicine', 'North Shore University Hospital']

best_avg_ranking = float('inf')
best_weights = None

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 3 weights are 1
    if sum(weights) < 5:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the average ranking for the selected locations
    avg_ranking = df_copy.loc[df_copy['Temp Locations 2'].isin(locations), 'ranking'].mean()
    # if this is the best average ranking we've found so far, store the weights
    if avg_ranking < best_avg_ranking:
        best_avg_ranking = avg_ranking
        best_weights = weights

# print the best weights and the corresponding average ranking
print(f"Best weights: {best_weights}")
print(f"Average ranking for selected locations using best weights: {best_avg_ranking}")

# apply the best weights to the DataFrame
df_best = df.copy()
for column, weight in zip(score_columns, best_weights):
    df_best[column] *= weight
# calculate the new score
df_best['score'] = df_best[score_columns].sum(axis=1)
# rank the rows by the new score
df_best['ranking'] = df_best['score'].rank(method='min', ascending=False)
# print the rankings of the selected locations
for location in locations:
    rank = df_best.loc[df_best['Temp Locations 2'] == location, 'ranking'].iloc[0]
    print(f"Rank of {location} with best weights: {rank}")



Best weights: (1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
Average ranking for selected locations using best weights: 56.666666666666664
Rank of University of Alabama Medical Center with best weights: 21.0
Rank of Singing River, Mississippi with best weights: 1.0
Rank of Bringham Young, Massachusetts with best weights: 41.0
Rank of Kansas University Medical Centre (KUMC) with best weights: 67.0
Rank of University of Cincinnati with best weights: 65.0
Rank of Houston Methodist Hospital with best weights: 118.0
Rank of UVA Health Center with best weights: 114.0
Rank of Washington University School of Medicine with best weights: 59.0
Rank of North Shore University Hospital with best weights: 24.0


In [10]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

# list of locations of interest
locations = ['University of Alabama Medical Center', 'Singing River, Mississippi', 'Bringham Young, Massachusetts',
             'Kansas University Medical Centre (KUMC)', 'University of Cincinnati', 'Houston Methodist Hospital',
             'UVA Health Center', 'Washington University School of Medicine', 'North Shore University Hospital']

best_weights = None

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 3 weights are 1
    if sum(weights) < 3:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the rankings of the selected locations
    location_rankings = df_copy.loc[df_copy['Temp Locations 2'].isin(locations), 'ranking']
    # if any location is ranked over 100, skip this combination of weights
    if any(rank > 100 for rank in location_rankings):
        continue
    # if this is the first weights configuration that satisfies all conditions or if the maximum rank is smaller than the previous best, update the best weights
    if best_weights is None or location_rankings.max() < df_best.loc[df_best['Temp Locations 2'].isin(locations), 'ranking'].max():
        best_weights = weights
        df_best = df_copy

# print the best weights
print(f"Best weights: {best_weights}")

# print the rankings of the selected locations
for location in locations:
    rank = df_best.loc[df_best['Temp Locations 2'] == location, 'ranking'].iloc[0]
    print(f"Rank of {location} with best weights: {rank}")


Best weights: None
Rank of University of Alabama Medical Center with best weights: 21.0
Rank of Singing River, Mississippi with best weights: 1.0
Rank of Bringham Young, Massachusetts with best weights: 41.0
Rank of Kansas University Medical Centre (KUMC) with best weights: 67.0
Rank of University of Cincinnati with best weights: 65.0
Rank of Houston Methodist Hospital with best weights: 118.0
Rank of UVA Health Center with best weights: 114.0
Rank of Washington University School of Medicine with best weights: 59.0
Rank of North Shore University Hospital with best weights: 24.0


In [6]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

# list of locations of interest
locations = ['University of Alabama Medical Center', 'Singing River, Mississippi',
             'Kansas University Medical Centre (KUMC)', 'Houston Methodist Hospital',
               'North Shore University Hospital']

best_avg_ranking = float('inf')
best_weights = None

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 3 weights are 1
    if sum(weights) < 7:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the average ranking for the selected locations
    avg_ranking = df_copy.loc[df_copy['Temp Locations 2'].isin(locations), 'ranking'].mean()
    # if this is the best average ranking we've found so far, store the weights
    if avg_ranking < best_avg_ranking:
        best_avg_ranking = avg_ranking
        best_weights = weights

# print the best weights and the corresponding average ranking
print(f"Best weights: {best_weights}")
print(f"Average ranking for selected locations using best weights: {best_avg_ranking}")

# apply the best weights to the DataFrame
df_best = df.copy()
for column, weight in zip(score_columns, best_weights):
    df_best[column] *= weight
# calculate the new score
df_best['score'] = df_best[score_columns].sum(axis=1)
# rank the rows by the new score
df_best['ranking'] = df_best['score'].rank(method='min', ascending=False)
# print the rankings of the selected locations
for location in locations:
    rank = df_best.loc[df_best['Temp Locations 2'] == location, 'ranking'].iloc[0]
    print(f"Rank of {location} with best weights: {rank}")



Best weights: (1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1)
Average ranking for selected locations using best weights: 13.0
Rank of University of Alabama Medical Center with best weights: 13.0
Rank of Singing River, Mississippi with best weights: 1.0
Rank of Kansas University Medical Centre (KUMC) with best weights: 8.0
Rank of Houston Methodist Hospital with best weights: 27.0
Rank of North Shore University Hospital with best weights: 16.0


In [8]:
top_20_locations = df_best.sort_values(by='ranking', ascending=True).head(30)
print("\nTop 20 locations:")
print(top_20_locations['Temp Locations 2'])


Top 20 locations:
3                                Ocean Springs Hospital
4                            Singing River, Mississippi
10                               University of Arkansas
6              University of Mississippi Medical Center
38                 Vanderbilt University Medical Center
39                             Vanderbilt_University MC
40                       Le Bonheur Children's Hospital
86              Kansas University Medical Centre (KUMC)
16                CHRISTUS Good Shepherd Medical Center
20                             Venkatakrishna Kakkilaya
19                      Methodist Dallas Medical Center
18                  Parkland Health and Hospital System
0                     USA Children's & Women's Hospital
1                  University of Alabama Medical Center
2     University of South Alabama - USA Children's a...
77                       Mount Sinai School of Medicine
76            Feinstein Institutes for Medical Research
75                           

127.0.0.1 - - [25/Jul/2023 23:50:16] "GET /result HTTP/1.1" 304 -


['0', '3', '4', '5', '9', '10', '11', '12']
TIME 1
TIME 2
TIME 3
TIME 4
TIME 5
WEIGHT LIST [0, 3, 4, 5, 9, 10, 11, 12]
TIME 6
TIME 7


127.0.0.1 - - [25/Jul/2023 23:50:46] "POST /checkboxes HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2023 23:50:46] "GET /result HTTP/1.1" 200 -


TIME 8


...

TIME 8
['0', '3', '4', '5', '9', '10', '11', '12']
TIME 1
TIME 2
TIME 3
TIME 4
TIME 5
WEIGHT LIST [0, 3, 4, 5, 9, 10, 11, 12]
TIME 6
TIME 7


127.0.0.1 - - [25/Jul/2023 23:54:54] "POST /checkboxes HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2023 23:54:54] "GET /result HTTP/1.1" 200 -


TIME 8
['0', '12']
TIME 1
TIME 2
TIME 3
TIME 4
TIME 5
WEIGHT LIST [0, 12]
TIME 6
TIME 7


127.0.0.1 - - [26/Jul/2023 00:02:06] "POST /checkboxes HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 00:02:06] "GET /result HTTP/1.1" 200 -


TIME 8


In [11]:
import pandas as pd
from itertools import product

df = pd.read_csv('rankedlocations.csv')

# create a list of all the columns that contribute to the score
score_columns = ['normalized_covid', 'normalized_sepsis', 'normalized_drowning', 'normalized_vaccination', 
                 'normalized_flu', 'normalized_pneumonia', 'normalized_literacy', 'normalized_seniors',
                 'normalized_smoking', 'normalized_copd', 'normalized_health_insurance', 'normalized_incomes']

# list of locations of interest
locations = ['University of Alabama Medical Center', 'Singing River, Mississippi',
             'Kansas University Medical Centre (KUMC)', 'Houston Methodist Hospital',
               'North Shore University Hospital']

best_weights = None

# iterate over all possible combinations of 1s and 0s
for weights in product([0, 1], repeat=len(score_columns)):
    # Skip if less than 3 weights are 1
    
    if sum(weights) < 7:
        continue
    # create a copy of the original DataFrame
    df_copy = df.copy()
    # for each column, multiply it by the corresponding weight
    for column, weight in zip(score_columns, weights):
        df_copy[column] *= weight
    # calculate the new score
    df_copy['score'] = df_copy[score_columns].sum(axis=1)
    # rank the rows by the new score
    df_copy['ranking'] = df_copy['score'].rank(method='min', ascending=False)
    # get the rankings of the selected locations
    location_rankings = df_copy.loc[df_copy['Temp Locations 2'].isin(locations), 'ranking']
    # if any location is ranked over 100, skip this combination of weights
    if any(rank > 20 for rank in location_rankings):
        continue
    # if this is the first weights configuration that satisfies all conditions or if the maximum rank is smaller than the previous best, update the best weights
    if best_weights is None or location_rankings.max() < df_best.loc[df_best['Temp Locations 2'].isin(locations), 'ranking'].max():
        best_weights = weights
        df_best = df_copy

# print the best weights
print(f"Best weights: {best_weights}")

# print the rankings of the selected locations
for location in locations:
    rank = df_best.loc[df_best['Temp Locations 2'] == location, 'ranking'].iloc[0]
    print(f"Rank of {location} with best weights: {rank}")


Best weights: None
Rank of University of Alabama Medical Center with best weights: 13.0
Rank of Singing River, Mississippi with best weights: 1.0
Rank of Kansas University Medical Centre (KUMC) with best weights: 8.0
Rank of Houston Methodist Hospital with best weights: 23.0
Rank of North Shore University Hospital with best weights: 16.0


In [1]:
import folium
import folium.plugins as plugins
import datetime

m = folium.Map([42.5, -100], zoom_start=4)

# GeoJson data where each feature has a property 'time' containing the date of the data.
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [-80, 35]
        },
        'properties': {
            'time': datetime.datetime(2017, 1, 1).isoformat(),
            'style': {'color': 'red'}
        }
    },
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [-80, 36]
        },
        'properties': {
            'time': datetime.datetime(2018, 1, 1).isoformat(),
            'style': {'color': 'blue'}
        }
    },
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [-80, 37]
        },
        'properties': {
            'time': datetime.datetime(2019, 1, 1).isoformat(),
            'style': {'color': 'green'}
        }
    }
]

plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1Y',  # The period denotes the length of time between steps in the animation. 'P1Y' means a period of 1 year.
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True
).add_to(m)

m.save('timeslider.html')
